In [1]:
import os
from google.colab import drive

# 1. 드라이브 연결
drive.mount('/content/drive')

# 2. 파일이 있는 폴더로 이동 (이미지의 경로 기준)
os.chdir('/content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016-ChestOnly')

print(f"현재 위치: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 위치: /content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016-ChestOnly


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# 1. PKL 로드
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/HAR_data/RealWorld2016/RealWorld2016_Master_Sync.pkl")

WINDOW_SIZE = 100  # 2초 (50Hz)
STEP = 50          # 50% 겹침
FEATURE_COLS = [c for c in df.columns if ('acc_' in c or 'gyr_' in c) and 'chest' in c]
print(f"선택된 컬럼 수: {len(FEATURE_COLS)}") # 6

# 라벨 인코딩
le = LabelEncoder()
df['label'] = le.fit_transform(df['activity'])
print("Classes:", le.classes_)

# Windowing 함수 (수정됨: Instance Norm 추가)
def create_dataset(df, window_size, step):
    Xs, ys = [], []
    grouped = df.groupby(['subject_id', 'activity', 'trial'])

    for _, group in grouped:
        data = group[FEATURE_COLS].values
        label = group['label'].values[0]

        # 슬라이딩 윈도우
        for i in range(0, len(data) - window_size, step):
            window = data[i:i+window_size]

            Xs.append(window)
            ys.append(label)

    return np.array(Xs), np.array(ys)

# Train / Test 분리
test_subjects = [6, 11, 12]
test_df = df[df['subject_id'].isin(test_subjects)].copy()
train_df = df[~df['subject_id'].isin(test_subjects)].copy() # ~는 반대(NOT)를 의미
print(f"Train Subjects: {train_df['subject_id'].unique()}")
print(f"Test Subjects : {test_df['subject_id'].unique()}")

print("⏳ 윈도우 자르기 시작... (조금 걸림)")
X_train, y_train = create_dataset(train_df, WINDOW_SIZE, STEP)
X_test, y_test = create_dataset(test_df, WINDOW_SIZE, STEP)

print(f"✅ 학습 데이터: {X_train.shape}")
print(f"✅ 평가 데이터: {X_test.shape}")

# 저장
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
print("💾 전처리 및 저장 완료!")

선택된 컬럼 수: 6
Classes: ['climbingdown' 'climbingup' 'jumping' 'lying' 'running' 'sitting'
 'standing' 'walking']
Train Subjects: [ 1  2  3  4  5  7  8  9 10 13 14 15]
Test Subjects : [ 6 11 12]
⏳ 윈도우 자르기 시작... (조금 걸림)
✅ 학습 데이터: (53459, 100, 6)
✅ 평가 데이터: (12915, 100, 6)
💾 전처리 및 저장 완료!


In [3]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=1d6ad3923e1dd36e9aa239516a5e95fc9ffd4116ccd68e35e7fe2fbd9770cebc
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=c923f226e4c52d930abfbfa17be6ffa0c92b34bc940385439da83581bce9251d
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


In [4]:
!python train.py

[01/50] Train Loss: 0.866 | F1: 0.8253  |  Test F1: 0.6398 (Best: 0.6398) Saved Best Model
[02/50] Train Loss: 0.596 | F1: 0.9269  |  Test F1: 0.7005 (Best: 0.7005) Saved Best Model
[03/50] Train Loss: 0.545 | F1: 0.9394  |  Test F1: 0.6544 (Best: 0.7005) 
[04/50] Train Loss: 0.518 | F1: 0.9466  |  Test F1: 0.6943 (Best: 0.7005) 
[05/50] Train Loss: 0.491 | F1: 0.9525  |  Test F1: 0.6883 (Best: 0.7005) 
[06/50] Train Loss: 0.482 | F1: 0.9532  |  Test F1: 0.6898 (Best: 0.7005) 
[07/50] Train Loss: 0.471 | F1: 0.9562  |  Test F1: 0.6886 (Best: 0.7005) 
[08/50] Train Loss: 0.460 | F1: 0.9592  |  Test F1: 0.6873 (Best: 0.7005) 
[09/50] Train Loss: 0.453 | F1: 0.9609  |  Test F1: 0.6828 (Best: 0.7005) 
[10/50] Train Loss: 0.442 | F1: 0.9632  |  Test F1: 0.6421 (Best: 0.7005) 
[11/50] Train Loss: 0.440 | F1: 0.9635  |  Test F1: 0.6412 (Best: 0.7005) 
[12/50] Train Loss: 0.430 | F1: 0.9662  |  Test F1: 0.6414 (Best: 0.7005) 
[13/50] Train Loss: 0.423 | F1: 0.9681  |  Test F1: 0.7134 (Best: 0.

In [5]:
!python visualize.py

--------------------------------------------------------------------------------
Running Visualization...
Loading weights from: /content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016-ChestOnly/RealWorld2016-ChestOnly_ASF.pth
--------------------------------------------------------------------------------
✅ Model weights loaded successfully!
--------------------------------------------------------------------------------
Evaluating Model Efficiency...
1. Parameters       : 0.0957 M
2. FLOPs / sample   : 5.249 M
3. Infer time       : 1.53 ms/sample
--------------------------------------------------------------------------------
Extracting features and predictions...
Generating Confusion Matrix...

Classification Report
              precision    recall  f1-score   support

climbingdown     0.8688    0.7310    0.7940      1476
  climbingup     0.6101    0.6262    0.6180      1704
     jumping     0.9686    0.9343    0.9512       198
       lying     0.9278    0.9761    0.9513      1921
